# This keyword
A tutorial on the use of the *this* keyword in JavaScript

## Top Level 
At the very top level the _this_ keyword refers to the **global** object in a Node REPL or the **window** object in the browser. In a Node module it refers to **module.exports**.

In [3]:
// Top level Node REPL
this === global;

true

In [ ]:
// Top level browser
this === window; //only works in a browser

In [4]:
// Top level Node module
this === module.exports;

false

## Plain Functions
In plain functions, _this_ behaves differently depending on `'strict mode'`.
When strict mode is off, _this_ refers to the global object.

In [5]:
function notInStrictMode(){
    return this === global;
}
notInStrictMode();

true

When strict mode is on, _this_ is undefined. This is the same
whether the strict mode is defined in the function or in the 
function's context.

In [6]:
function inStrictMode(){
    'use strict';
    return this === undefined;
}
inStrictMode();

true